In [ ]:
"""This ipynb helps convert the instance prediction to semantic prediction"""

In [ ]:
import os
import numpy as np
from prostate_dataset import ProstateDataset

data_path = '/data/wenyuan/Path_R_CNN/Data_Pre_Processing'
dirname = os.path.join(data_path, 'cedars-224')
data = ProstateDataset(dirname)

In [ ]:
_, mask, class_ids, scores = data.read_mask_rcnn_output(12)

In [ ]:
def prediction_2_sementic(mask, class_ids, scores):
    """ convert maskrcnn prediction to 1 single sementic mask
    """
    mask_map = {} # create a probability dict for each class
    for index, label in enumerate(class_ids):
        mask_map[str(label)] = np.maximum(mask_map[str(label)], scores[index] * mask[:, :, index])\
        if str(label) in mask_map.keys() else scores[index] * mask[:, :, index]
    
    ## convert to h * w * num_classes probability map
    h, w, d = mask.shape
    for i in range(4):
        try:
            sementic_mask = np.concatenate((sementic_mask, np.expand_dims(mask_map[str(i)], axis = -1)),axis = -1)\
            if (i != 0) else np.expand_dims(mask_map[str(i)], axis = -1)
        except KeyError:
            sementic_mask = np.concatenate((np.zeros((h, w, 1)), mask_map[str(i)]),axis = -1)\
            if (i != 0) else np.zeros((h, w, 1))
            
    sementic_mask_res = np.argmax(sementic_mask, axis = -1)
    return sementic_mask_res


In [ ]:
mask = prediction_2_sementic(mask, class_ids, scores)
mask_decode = data.convert_mat_annotations_to_png(mask)
data.show_ann_png(mask_decode)